# Question 3

#### Imports

In [31]:
import os

import numpy as np

from classify.data.responses import responses_to_traffic_array
from classify.data.traffic import load_traffic
from classify.scenario.bridge import healthy_damage, pier_disp_damage
from classify.scenario.traffic import normal_traffic
from model.bridge import Point
from model.bridge.bridge_705 import bridge_705_3d, bridge_705_med_config
from model.response import ResponseType
from util import resize_units

#### Config

In [32]:
DATA_PATH = "/Users/jeremy/Desktop/mesh-med-600-mar/"
IMAGE_PATH = "/Users/jeremy/Desktop/saved-images/"
c = bridge_705_med_config(bridge_705_3d)
# Set the directory of where to save/load responses.
c.root_generated_data_dir = os.path.join(DATA_PATH, c.root_generated_data_dir)

INFO: Loaded vehicle data from /Users/jeremy/cs/bridge-dss/data/a16-data/a16.csv in 0.24s
WARN: Vehicle PDF sums to 99.5, adjusted to sum to 1


#### Aliases and sensors

In [33]:
# Short aliases for response types.
rt_y = ResponseType.YTranslation
rt_s = ResponseType.Strain

# Create functions to resize, and unit strings, for each response type.
resize_y, units_y = resize_units(rt_y.units())
resize_s, _ = resize_units(rt_s.units())

sensor_point = Point(x=21, y=0, z=-8.4)  # A sensor point to investigate.
sensor_point = Point(x=33, y=0, z=-4)  # A sensor point to investigate.

#### Traffic data

In [34]:
total_mins = 24
total_seconds = total_mins * 60
traffic_scenario = normal_traffic(c=c, lam=5, min_d=2)
traffic_sequence_0, traffic_0, traffic_array_0 = load_traffic(
    c=c,
    traffic_scenario=traffic_scenario,
    max_time=total_seconds,
)

/Users/jeremy/Desktop/mesh-med-600-mar/generated-data/bridge-705-3d/healthy/traffic/normal-lam-5-600-1440-0,01.npy


#### Setup each damage scenario.

In [35]:
damage_scenarios = [healthy_damage, pier_disp_damage([(5, 3 / 1000)])]
traffic_arrays = [traffic_array_0, traffic_array_0]
damage_names = ["Healthy", "Pier settlement by 3 mm"]
response_types = [rt_y, rt_s]

#### Collect responses under each damage scenario.

In [36]:
responses = [[None for _ in response_types] for _ in damage_scenarios]
for d_i, (damage_scenario, traffic_array) in enumerate(zip(damage_scenarios, traffic_arrays)):
    for r_i, response_type in enumerate(response_types):
        responses[d_i][r_i] = responses_to_traffic_array(
            c=damage_scenario.use(c)[0],
            traffic_array=traffic_array,
            response_type=response_type,
            damage_scenario=damage_scenario,
            points=[sensor_point],
        ).T[0]  # Responses from a single point.
        # Resize responses into mm and strain.
        if response_type == rt_y:
            responses[d_i][r_i] = resize_y(responses[d_i][r_i])
        elif response_type == rt_s:
            responses[d_i][r_i] = resize_s(responses[d_i][r_i])
responses = np.array(responses)
responses.shape

(144001, 2400)
(2400, 1)
(144001, 2400)
(2400, 1)
(144001, 2400)
(2400, 1)
(144001, 2400)uilt FEMResponses in 0.01s, (ResponseType.YTranslation)
(2400, 1)


(2, 2, 144001)

#### Removal of temperature effect

In [37]:
downsample = 100
data = []
datetimes = []
for month_i, day_i in itertools.product([7, 8, 9, 10, 11, 12], np.arange(1, 30 + 1)):
    day_str, month_str = str(day_i), str(month_i)
    if len(day_str) == 1: day_str = "0" + day_str
    if len(month_str) == 1: month_str = "0" + month_str
    temp_i, temp_j = ij(temps_2019, f"2019-{month_str}-{day_str}T00:00", f"2019-{month_str}-{day_str}T23:59")
    day_temp = np.mean(temps_2019["temp"][temp_i:temp_j])
    print(f"Day, month = {day_str}, {month_str}, Mean day temperature = {day_temp:.3f}", end="\r")
    rm_year_s = lr_s_2018.predict([[day_temp]])[0]
    rm_year_y = lr_y_2018.predict([[day_temp]])[0]
#     print(f"Y/S effect from 2018 = {rm_year_y:.3f}, {rm_year_s}")
    datetimes.append(datetime.fromisoformat(f"2019-{month_str}-{day_str}T00:00"))
    data.append([])
    for (traffic_y, traffic_s), damage_name in zip(responses, damage_names):
        temp_y = temperature.apply(effect=effect_2019_y[temp_i:temp_j], responses=traffic_y)
        temp_s = temperature.apply(effect=effect_2019_s[temp_i:temp_j], responses=traffic_y)
        _rm_24h_y, rm_24h_y = remove_sampled(24, traffic_y + temp_y)
        _rm_24h_s, rm_24h_s = remove_sampled(24, traffic_s + temp_s)
        signal_s = traffic_s + temp_s - rm_24h_s - rm_year_s
        signal_y = traffic_y + temp_y - rm_24h_y - rm_year_y
#         signal_s = traffic_s + temp_s - rm_year_s
#         signal_y = traffic_y + temp_y - rm_year_y
        data[-1].append(np.array([signal_y, signal_s]))
data = np.array(data)
# data = np.resize(data, (data.shape[1], data.shape[2], data.shape[0], data.shape[3]))
data.shape

NameError: name 'itertools' is not defined

#### 6 month daily prediction

In [ ]:
healthy_y, pier_y = [], []
for day_i in range(data.shape[0]):
    healthy_y.append(np.mean(data[day_i][0][0]))
    pier_y.append(np.mean(data[day_i][1][0]))
healthy_y, pier_y = np.array(healthy_y), np.array(pier_y)

#### 6 month 14 day prediction

In [ ]:
days = 14  # Number of days for robust estimator.
healthy_robust, pier_robust = [], []
for i in range(days, len(healthy_y)):
    healthy_robust.append(np.mean(healthy_y[i-days:i]))
    pier_robust.append(np.mean(pier_y[i-days:i]))
healthy_robust, pier_robust = np.array(healthy_robust), np.array(pier_robust)

#### Plotting prediction

In [ ]:
plt.plot(datetimes, healthy_y)
plt.plot(datetimes, pier_y)
plt.plot(datetimes[days:], healthy_robust)
plt.plot(datetimes[days:], pier_robust)

#### Classify based on accuracy

In [ ]:
thresholds = np.arange(start=1, stop=-3, step=-0.01)
count_hs, count_ps = [], []
for threshold in thresholds:
    count_h, count_p = 0, 0
    for day_i in range(len(healthy_robust)):
        if healthy_robust[day_i] < threshold:
            count_h += 1
        if pier_robust[day_i] < threshold:
            count_p += 1
    count_hs.append(count_h)
    count_ps.append(count_p)
count_hs, count_ps = np.array(count_hs), np.array(count_ps)
#     print(f"Threshold = {threshold}, healthy: {count_h / len(healthy_robust)}, pier: {count_p / len(healthy_robust)}")

#### Plot threshold.

In [ ]:
tp, fn = count_ps, len(pier_robust) - count_ps
tn, fp = len(pier_robust) - count_hs, count_hs
# plt.plot(thresholds, tp / len(pier_robust), label="True negative", color="tab:blue", lw=4)
# plt.plot(thresholds, fn / len(pier_robust), label="False positive", color="tab:red", lw=4)
# plt.plot(thresholds, tn / len(pier_robust), label="True positive", color="tab:green", lw=4)
# plt.plot(thresholds, fp / len(pier_robust), label="False negative", color="tab:orange", lw=4)
plt.plot(thresholds, (tp) / (fn + tp), label="Sensitivity", color="tab:green", lw=4)
plt.plot(thresholds, (tn) / (fp + tn), label="Specificity", color="tab:blue", lw=4)
plt.plot(thresholds, (fn) / (fn + tp), label="Miss rate", color="tab:orange", lw=4)
plt.plot(thresholds, (fp) / (fp + tn), label="False positive rate", color="tab:red", lw=4)
plt.plot(thresholds, (tn + tp) / (fn + fp + tn + tp), label="Accuracy", color="black", lw=4)
# plt.plot(thresholds, (tp) / (tp + fn), label="Accuracy")
plt.ylabel("Rate")
plt.xlabel("Y translation threshold")
plt.legend(loc="upper right")
plt.title(f"Classification as a function of Y translation threshold\nJuly - Dec, at sensor at x = {sensor_point.x:.1f} m, z = {sensor_point.z:.1f} m")
plt.savefig(os.path.join(IMAGE_PATH, f"fp-fp-tn-tp-{sensor_point.x:.0f}-{sensor_point.z:.0f}.pdf"))